In [160]:
%%writefile /content/agents/student_agent.py
from typing import List, Dict, Any
from scipy.stats._multivariate import special_ortho_group_frozen
from agents import HouseOwnerAgent, CompanyAgent
from communication import NegotiationMessage

class MyACMEAgent(HouseOwnerAgent):
    def __init__(self, role: str, budget_list: List[Dict[str, Any]]):
        super(MyACMEAgent, self).__init__(role, budget_list)
        self.contracts_signed = {}  # Stores finalized contracts
        self.bidders_per_item = {}  # Track number of bidders in auctions
        self.previous_partner_offers = {}  # Store incoming offers per round
        self.own_offers = {}  # Store outgoing offers per round

    def propose_item_budget(self, auction_item: str, auction_round: int) -> float:
        # Increase budget offer per round, capped at 100% of budget
        budget = self.budget_dict[auction_item]
        offer = budget * min(1.0, 0.5 + 0.25 * auction_round)
        return offer

    def notify_auction_round_result(self, auction_item: str, auction_round: int, responding_agents: List[str]):
        self.bidders_per_item[auction_item] = len(responding_agents)

    def provide_negotiation_offer(self, negotiation_item: str, partner_agent: str, negotiation_round: int) -> float:
        budget = self.budget_dict[negotiation_item]
        self.own_offers.setdefault(negotiation_item, {}).setdefault(partner_agent, [])

        min_partner_offer = None
        offer = 0

        if negotiation_round == 0:
            offer = budget * 0.5  # Start negotiation at 50% of budget
        else:
            # Determine lowest offer received from any partner
            partner_offers = self.previous_partner_offers.get(negotiation_item, {})
            for offers in partner_offers.values():
                if len(offers) >= negotiation_round:
                    last_offer = offers[negotiation_round - 1]
                    if min_partner_offer is None or last_offer < min_partner_offer:
                        min_partner_offer = last_offer

            if min_partner_offer is not None:
                prev_offer = self.own_offers[negotiation_item][partner_agent][-1]
                diff = min_partner_offer - prev_offer

                # If concession is small, match the partner’s offer
                if diff <= 0.1 * prev_offer:
                    offer = min(budget, min_partner_offer)
                    print(f"[ACME] ✅ Diff too small → matching firm's offer: {offer:.2f}")
                else:
                    offer = min(budget, min_partner_offer * 0.9)
            else:
                # Increase own offer moderately
                prev_offer = self.own_offers[negotiation_item][partner_agent][-1]
                offer = min(budget, prev_offer * 1.1)

        self.own_offers[negotiation_item][partner_agent].append(offer)
        print(f"[ACME] Round {negotiation_round} | {negotiation_item} → offer to {partner_agent}: {offer:.2f}")
        return offer

    def notify_partner_response(self, response_msg: NegotiationMessage) -> None:
        item = response_msg.negotiation_item
        partner = response_msg.sender
        self.previous_partner_offers.setdefault(item, {}).setdefault(partner, []).append(response_msg.offer)

    def notify_negotiation_winner(self, negotiation_item: str, winning_agent: str, winning_offer: float) -> None:
        self.contracts_signed[negotiation_item] = {
            "agent": winning_agent,
            "price": winning_offer
        }

    def __del__(self):
        print("\n=== [ACME METRICS]")
        total_budget = sum(self.budget_dict.values())
        total_spent = sum(c["price"] for c in self.contracts_signed.values())
        remaining = total_budget - total_spent
        print(f" Contracts completed: {len(self.contracts_signed)}/{len(self.budget_dict)}")
        print(f" Total budget: {total_budget}")
        print(f" Total spent: {total_spent}")
        print(f" Remaining funds: {remaining}")
        for item, c in self.contracts_signed.items():
            print(f"   • {item}: {c['price']} → {c['agent']}")
        print()
        if len(self.contracts_signed) == len(self.budget_dict):
            firm_profit = MyCompanyAgent.total_firm_profit
            if firm_profit > 0:
                ratio = remaining / firm_profit
                print(f"📈 Utility (ACME remaining / total company profit): {ratio:.2f}")
            else:
                print(f"📈 Utility: {remaining:.2f} (ACME remaining funds, company profit = 0)")


class MyCompanyAgent(CompanyAgent):
    total_firm_profit = 0.0

    def __init__(self, role: str, specialties: List[Dict[str, Any]]):
        super(MyCompanyAgent, self).__init__(role, specialties)
        self.total_profit = 0.0
        self.contracts = []  # Store won contracts
        self.offer_history = {}

    def decide_bid(self, auction_item: str, auction_round: int, item_budget: float) -> bool:
        cost = self.specialties[auction_item]
        # If company has no contracts yet, be more aggressive
        min_price = cost * 0.7 if len(self.contracts) == 0 else cost * 1.1
        return item_budget >= min_price

    def notify_won_auction(self, auction_item: str, auction_round: int, num_selected: int):
        pass

    def respond_to_offer(self, initiator_msg: NegotiationMessage) -> float:
        item = initiator_msg.negotiation_item
        cost = self.specialties[item]
        offer = initiator_msg.offer
        self.offer_history.setdefault(item, [])

        if initiator_msg.round == 0:
            # Accept if profitable enough, otherwise counter with margin
            if offer >= cost * 1.23:
                response = offer
            else:
                response = max(cost * 1.23, offer * 1.25)
        else:
            last_response = self.offer_history[item][-1]
            response = max(cost, last_response * 0.95)  # Gradual concession

        self.offer_history[item].append(response)
        print(f"[{self.name}] Round {initiator_msg.round} | {item} ← responds with: {response:.2f}")
        return response

    def notify_contract_assigned(self, construction_item: str, price: float) -> None:
        if any(item == construction_item for item, _, _ in self.contracts):
            return
        cost = self.specialties[construction_item]
        profit = price - cost
        self.total_profit += profit
        MyCompanyAgent.total_firm_profit += profit
        self.contracts.append((construction_item, price, profit))
        print(f"[{self.name}] ✅ Contract for {construction_item} @ {price:.2f}")

    def notify_negotiation_lost(self, construction_item: str) -> None:
        pass

    def __del__(self):
        if self.contracts:
            print(f"\n=== [{self.name} METRICS]")
            print(f" Contracts won: {len(self.contracts)}")
            for item, price, profit in self.contracts:
                print(f"   • {item}: earned {price:.2f} (profit: {profit:.2f})")
            print(f" Total profit: {self.total_profit:.2f}")

Overwriting /content/agents/student_agent.py


In [161]:
!python environment.py

[Auction stage]
Item  :  structural design / 2500.0
    agent _bids :  dict_values([False, False, False])
responding agent are :  []
#### House Building Environment ####

[Auction stage]
Item  :  structural design / 3750.0
    agent _bids :  dict_values([False, True, True])
responding agent are :  ['MyCompanyAgent_B', 'MyCompanyAgent_F']
2025-05-18 12:20:56,370 - environment - INFO - [NOTIFICATION] Companies ['MyCompanyAgent_B', 'MyCompanyAgent_F'] have accepted construction item structural design at price: 3750.0
#### House Building Environment ####

[Auction stage]
Item  :  structure building / 5000.0
    agent _bids :  dict_values([False, False, False, False, True, False])
responding agent are :  ['MyCompanyAgent_E']
2025-05-18 12:20:56,370 - environment - INFO - [NOTIFICATION] Companies ['MyCompanyAgent_E'] have accepted construction item structure building at price: 5000.0
#### House Building Environment ####

[Auction stage]
Item  :  electrics and plumbing / 2000.0
    agent _bid